In [272]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ, final_exponentiate, FQ12
import numpy as np
import galois   

In [273]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [274]:
# This script demonstrates the polynomial evaluations with powers of tau
# we see that evaluating a polyomial at tau is the same as computing the inner product of the polynomial coefficients and the powers of tau

tau_testing = GF(5)

# (4x^3 + 2x^2 + 3x + 1) * (x+2)
polynomial = galois.Poly([4,2,3,1], field=GF) * galois.Poly([1,2], field=GF)
print(f"Polynomial: {polynomial}")  # Outputs: 4x^4 + 10x^3 + 7x^2 + 7x + 2

# Evaluate the polynomial at tau.
p_eval_at_tau = polynomial(tau_testing)
print(f"Polynomial evaluated at tau: {p_eval_at_tau}") # Outputs: 3962
p_eval_at_tau_G1 = multiply(G1, int(p_eval_at_tau))

def powers_of_tau_test(tau, degree):
    powers_of_tau_testing = []
    for i in range(degree+1):
        print(f"i = {i}, tau**i = {tau**i}")
        powers_of_tau_testing.append(multiply(G1, int(tau**i)))
    return powers_of_tau_testing

def compute_powers_of_tau_test(tau, degree):
    """Computes and prints the powers of tau up to the specified degree and returns them
    as elliptic curve points by multiplying each power by the base point G1."""
    powers_of_tau = []
    for i in range(degree + 1):
        tau_power = tau ** i
        tau_power_G1 = multiply(G1, int(tau_power))
        powers_of_tau.append(tau_power_G1)
        print(f"i = {i}, tau**i = {tau_power}, tau_power_G1 = {tau_power_G1}")
    return powers_of_tau


# create [τ^0 G1],[τ^1 G1][τ^2 G1]...[τ^degree +1 G1]
powers_of_tau_testing = compute_powers_of_tau_test(tau_testing, polynomial.degree)

# reverse the polynomial coefficients for 4x^4 + 10x^3 + 7x^2 + 7x + 2  to [2, 7, 7, 10, 4]
reversed_polynomial_coeffs = polynomial.coeffs[::-1]
print(f"Reversed polynomial coefficients: {reversed_polynomial_coeffs}")

def inner_product_test(powers_of_tau, coeffs):
    """Computes the elliptic curve point representing the inner product of the reversed polynomial coefficients
    and the powers of tau points."""
    accumulator = Z1
    for tau_power, coeff in zip(powers_of_tau, coeffs):
        term = multiply(tau_power, int(coeff))
        accumulator = add(accumulator, term)
    return accumulator

assert inner_product_test(powers_of_tau_testing, reversed_polynomial_coeffs) == p_eval_at_tau_G1, "doesn't match"

Polynomial: 4x^4 + 10x^3 + 7x^2 + 7x + 2
Polynomial evaluated at tau: 3962
i = 0, tau**i = 1, tau_power_G1 = (1, 2)
i = 1, tau**i = 5, tau_power_G1 = (10744596414106452074759370245733544594153395043370666422502510773307029471145, 848677436511517736191562425154572367705380862894644942948681172815252343932)
i = 2, tau**i = 25, tau_power_G1 = (20765039372871530718554589730410158162413780974122112544611863764810626751360, 2444183914824638066910831265243126275246160293098948571390980460351548298384)
i = 3, tau**i = 125, tau_power_G1 = (401978602110496381074453603102128333214265165139113426216634730340407351423, 12084390642472120358442724381421408623274349577217648002291952443608077749768)
i = 4, tau**i = 625, tau_power_G1 = (5527101163656315346960364963545370964626232799933256760464143148793261937308, 5380388670064483190479346872818688372396976240663815701649018878271631274675)
Reversed polynomial coefficients: [ 2  7  7 10  4]


In [275]:
# R1CS

x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

L = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

R = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

O = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [curve_order-2,1,0,3,0,curve_order-1,0,curve_order-2] # 113 - 2 = 111
    ]))

# m = number of columns
# n = number of rows
m = L.shape[1]
n = L.shape[0]
print(f"m={m}, n={n}")

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
print(f"witness= {type(witness)}")
print(f"generated L, R, O R1CS matrices and witness vector")
print(f"verifying that L.w * R.w == O.w")
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"
print(f"verified")

m=8, n=5
witness= <class 'galois.GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)'>
generated L, R, O R1CS matrices and witness vector
verifying that L.w * R.w == O.w
verified


In [276]:
# R1CS => QAP
target_coeffs = GF(np.array([1,2,3,4,5]))
target_polynomial = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4,5])).astype(int),  field=GF)

def matrix_to_vector_of_polynomials(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def interpolate_columns(column):
    return galois.lagrange_poly(target_coeffs, column)

# Once interpolation is finished, L, R, O = U.a, V.a, W.a
U_vector_of_poly_x = matrix_to_vector_of_polynomials(L)
V_vector_of_poly_x = matrix_to_vector_of_polynomials(R)
W_vector_of_poly_x = matrix_to_vector_of_polynomials(O)

def aggregate_vector_of_polynomials_with_witness(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

# This is a polynomial of the inner product of Matrix and Witness
Ua = aggregate_vector_of_polynomials_with_witness(U_vector_of_poly_x, witness)
Va = aggregate_vector_of_polynomials_with_witness(V_vector_of_poly_x, witness)
Wa = aggregate_vector_of_polynomials_with_witness(W_vector_of_poly_x, witness)

print(f"Ua= {U_vector_of_poly_x}")

# We can now create h(x)t(x)
hx, remainder = divmod(((Ua*Va) - Wa), target_polynomial)
assert remainder == 0, "remainder is not 0"
Ua * Va == Wa + hx * target_polynomial 
print(f"lhs = rhs so they are the same")

tau = GF(9)
Ua(tau) * Va(tau) == Wa(tau) + hx(tau) * target_polynomial(tau)
print(f"when evaluated at random number tau, theyare the same")

Ua= [Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904247810x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602831868x^2 + 4x, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(16416182153879456416684804308942956316411273300312025757773653139931856371713x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495614x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371725x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247789x + 10, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617))
 Poly(3648040478639879203707734290876212514758060

In [277]:
# Trusted Setup
from functools import reduce
tau = GF(9)

powers_of_tau_G1 = [multiply(G1, int(tau) ** i) for i in range(0,n)]
powers_of_tau_G2 = [multiply(G2, int(tau) ** i) for i in range(0,n)]
t_of_tau = target_polynomial(tau)
target_powers_of_tau_G1 = [multiply(G1, int(tau) ** i * int(t_of_tau)) for i in range(0,target_polynomial.degree)]

# Prover evaluate QAP at Tau
# This is not encrypted
Ua_tau = Ua(tau)
Va_tau = Va(tau)
Wa_tau = Wa(tau)
hx_tau = hx(tau)
tx_tau = target_polynomial(tau)
assert Ua_tau * Va_tau == Wa_tau + hx_tau * tx_tau, "Ua_tau * Va_tau != Wa_tau + hx_tau * tx_tau"

def inner_product(powers_of_tau, coeffs):
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_tau, coeffs)), Z1)

def inner_product_iterable(powers_of_tau, coeffs):
    print(len(powers_of_tau_G1), " eq? ", len(coeffs))
    result = Z1
    for i, (point, coeff) in enumerate(zip(powers_of_tau, coeffs)):
        print(f"Iteration: {i}, Coeff: {coeff}, Point: {point}")
        result = add(result, multiply(point, int(coeff)))
    return result
 
Ua_tau_G1 = inner_product_iterable(powers_of_tau_G1, Ua.coeffs[::-1])
Va_tau_G2 = inner_product_iterable(powers_of_tau_G2, Va.coeffs[::-1])
Wa_tau_G1 = inner_product_iterable(powers_of_tau_G1, Wa.coeffs[::-1])

ht_tau = inner_product_iterable(target_powers_of_tau_G1[0:hx.degree + 1], hx.coeffs[::-1])

A = pairing(Va_tau_G2, neg(Ua_tau_G1))
B = pairing(G2, add(Wa_tau_G1, ht_tau))
final_exponentiate(A * B) == FQ12.one()

5  eq?  5
Iteration: 0, Coeff: 21888242871839275222246405745257275088548364400416034343698204186575808494842, Point: (1, 2)
Iteration: 1, Coeff: 7296080957279758407415468581752425029516121466805344781232734728858602833414, Point: (1624070059937464756887933993293429854168590106605707304006200119738501412969, 3269329550605213075043232856820720631601935657990457502777101397807070461336)
Iteration: 2, Coeff: 3648040478639879203707734290876212514758060733402672390616367364429301414937, Point: (11744864753805541320111181058240501346617548818795024075343453303706881344088, 17221384307920533803726063289029126335755140866379809678854881527846933441919)
Iteration: 3, Coeff: 3648040478639879203707734290876212514758060733402672390616367364429301416197, Point: (16614481781419548918333901639087520596453061129401013916352956947099837560149, 6870598320119603643560241397292776091544668466385678757129689863273377343756)
Iteration: 4, Coeff: 729608095727975840741546858175242502951612146680534478123273472

True

In [278]:
# shift lhs and rhs by alpha and beta
# Trusted Setup
alpha = GF(2)
beta = GF(3)
AlphaG1 = multiply(G1, int(alpha))
BetaG2 = multiply(G2, int(beta))

# encrypted evaluation: shifts the U and V polynomial by alpha and beta, evaluates at powers of tau
# Trusted Setup completes this with unencrypted tau, alpha, beta values
powers_of_tau_C_G1 = []
for i in range(0, n-1):
    print(f"iteration = {i}, U_vector_of_poly_x[i] = {U_vector_of_poly_x[i]}, evaluated at tau = {U_vector_of_poly_x[i](tau)}, evaluated at tau**i = {U_vector_of_poly_x[i](tau) ** i} ")
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau**i)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau**i)
    w_eval_tau_i = W_vector_of_poly_x[i](tau**i)
    sum = int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)
    powers_of_tau_C_G1.append(multiply(G1, sum))

print(f"powers_of_tau_C_G1 = {powers_of_tau_C_G1}")
print(f"length: powers_of_tau_C_G1 = {len(powers_of_tau_C_G1)}")


Cprime = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(powers_of_tau_C_G1, witness)), Z1)
C_new = add(Cprime, ht_tau)

A_new = add(AlphaG1, Ua_tau_G1)
B_new = add(BetaG2, Va_tau_G2)

lhs = pairing(B_new, A_new)
rhs_1 = pairing(BetaG2,AlphaG1)
rhs_2 = pairing(G2,C_new)
rhs = rhs_1 * rhs_2

if eq(lhs, rhs):
    print("verified")
else:
    print("not verified")


iteration = 0, U_vector_of_poly_x[i] = 0, evaluated at tau = 0, evaluated at tau**i = 1 
iteration = 1, U_vector_of_poly_x[i] = 0, evaluated at tau = 0, evaluated at tau**i = 0 
iteration = 2, U_vector_of_poly_x[i] = 3648040478639879203707734290876212514758060733402672390616367364429301415936x^4 + 10944121435919637611123202872628637544274182200208017171849102093287904247810x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602831868x^2 + 4x, evaluated at tau = 21888242871839275222246405745257275088548364400416034343698204186575808495302, evaluated at tau**i = 99225 
iteration = 3, U_vector_of_poly_x[i] = 16416182153879456416684804308942956316411273300312025757773653139931856371713x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495614x^3 + 16416182153879456416684804308942956316411273300312025757773653139931856371725x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247789x + 10, evaluated at tau = 280, evaluate

In [279]:
# separate public and private inputs
# public input, computed by verifier
public_witness_length = 2
public_input = []

for i in range(public_witness_length):
    print("iteration", i, "witness: ", witness[i])
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau)
    w_eval_tau_i = W_vector_of_poly_x[i](tau)
    public_input.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))

# we now have 2 points representing Bui, Avi, Wvi. Let's squash the witness in and add to 1 point
public_input_w_witness = reduce(add,(multiply(point, int(coeff)) for point, coeff in zip(public_input, witness[:public_witness_length])), Z1)

# private input, computed by prover. Prover has access to witness and 
private_input = []
for i in range(public_witness_length,len(witness)):
    print("iteration", i, "witness: ", witness[i])
    u_eval_tau_i_beta = beta * U_vector_of_poly_x[i](tau)
    v_eval_tau_i_alpha = alpha * V_vector_of_poly_x[i](tau)
    w_eval_tau_i = W_vector_of_poly_x[i](tau)
    private_input.append(multiply(G1, int(u_eval_tau_i_beta + v_eval_tau_i_alpha + w_eval_tau_i)))

private_input_w_witness = reduce(add,(multiply(point, int(coeff)) for point, coeff in zip(private_input, witness[public_witness_length:])), Z1)

C_new = add(Cprime, ht_tau)



iteration 0 witness:  1
iteration 1 witness:  97
iteration 2 witness:  5
iteration 3 witness:  10
iteration 4 witness:  25
iteration 5 witness:  125
iteration 6 witness:  100
iteration 7 witness:  1250


In [280]:
# Introducing Gamma, Delta
gamma = GF(4)
delta = GF(5)
gamma_inv = np.reciprocal(gamma)
delta_inv = np.reciprocal(delta)

GammaG2 = multiply(G2, int(gamma))
DeltaG2 = multiply(G2, int(delta))

# powers of tau for public input from i = 0 to i = 1 for the 0, 1 pos of [1,out,x,y,...] 
public_witness_length = 2

# public
public_powers_of_tau_G1 = [multiply(point, int(gamma_inv)) for point in powers_of_tau_C_G1[:public_witness_length]]
gamma_pairing = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(public_powers_of_tau_G1, witness[0:public_witness_length])), Z1)


# private
private_powers_of_tau_G1 = [multiply(point, int(delta_inv)) for point in powers_of_tau_C_G1[public_witness_length+1:len(witness)]]
Cprime_delta = reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(private_powers_of_tau_G1, witness[public_witness_length:len(witness)])), Z1)


#ht_tau is from i = 0 -> m. Needs to change to l -> m
delta_ht_tau = multiply(ht_tau, int(delta_inv))
C_prime_new = add(Cprime_delta, ht_tau)
C_new = add(Cprime, ht_tau)

lhs = pairing(B_new, A_new)
rhs_1 = pairing(BetaG2,AlphaG1)
rhs_2 = pairing(GammaG2, gamma_pairing)
rhs_3 = pairing(DeltaG2, C_prime_new)
rhs = rhs_1 * rhs_2 * rhs_3

if eq(lhs, rhs):
    print("verified")
else:
    print("not verified")

not verified


In [281]:
import numpy as np
from scipy.interpolate import lagrange
x = np.array([1,2,3])
y = np.array([1,1,0])
print(lagrange(x, y))

      2
-0.5 x + 1.5 x
